引力波与暗物质晕交互可视化

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.cosmology import Planck18  # FlatLambdaCDM, FLRW cosmology with a cosmological constant and no curvature

In [ ]:
# set up other cosmological parameters
z_range = np.linspace(0.1,3,100)
comoving_dist = Planck18.comoving_distance(z_range)

# Distribution of dark matter halo
mass_halo = 1e12 * (1 + z_range)**2.5  # mass-z relation
density_profile = mass_halo / (4/3*np.pi*(100*3.086e21)**3)  # ~100kpc
# visualize
fig, ax = plt.subplots(figsize=(15,10))
im = ax.scatter(comoving_dist, np.zeros_like(z_range), c=density_profile, 
                cmap='viridis', s=mass_halo/1e11, alpha=0.6)
# GW path
wave_path = ax.plot(comoving_dist, 0.1*np.sin(10*comoving_dist.value), 
                    lw=2, color='cyan', label='GW path')

# 相位偏移
for z, d in zip(z_range[::10], comoving_dist[::10]):
    phase_shift = 1e-3 * (1 + z)**2  # 相位偏移模型
    ax.annotate(f'ΔΦ={phase_shift:.1e} rad', (d.value, 0), 
                rotation=45, fontsize=8)
plt.colorbar(im, label='Dark Matter Density (M⊙/pc³)')
ax.set_xlabel('Comoving Distance (Mpc)')
ax.set_title('GW Propagation through Dark Matter Halos')
plt.savefig('GW_lensing.png', dpi=300)

现有算法下双黑洞并合的SNR计算（可用于误报率计算）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform
from pycbc.noise import gaussian
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.filter import matched_filter
from pycbc.types import TimeSeries
from astropy.cosmology import Planck15 as cosmo

# 设置参数范围
total_mass_range = np.linspace(20, 100, 5)       # 太阳质量
q_range = np.linspace(1, 8, 5)                   # 质量比 q = m1/m2
z_range = np.linspace(0.3, 1.5, 4)               # 红移

# 设置采样率与数据长度
sample_rate = 4096
delta_t = 1.0 / sample_rate
data_len = 8      # 秒数
n_samples = int(data_len * sample_rate)

# 生成 PSD
psd = aLIGOZeroDetHighPower(n_samples//2 + 1, delta_f=1.0/data_len, low_freq_cutoff=20.0)

# 输出结果列表
results = []

for M in total_mass_range:
    for q in q_range:
        m1 = M * q / (1 + q)
        m2 = M / (1 + q)
        for z in z_range:
            # 计算红移距离
            dL = cosmo.luminosity_distance(z).to('Mpc').value

            # 生成波形（仅+极化）
            hp, hc = get_td_waveform(approximant="IMRPhenomPv2",
                                     mass1=m1, mass2=m2,
                                     distance=dL,
                                     inclination=0.0,
                                     delta_t=delta_t,
                                     f_lower=20.0)

            # 将波形裁剪为指定长度，并归一到数据长度
            hp = hp.crop(1, 1)
            hp.resize(n_samples)

            # 生成模拟噪声数据
            noise = gaussian.noise_from_psd(n_samples, delta_t, psd, seed=127)

            # 注入信号
            signal = noise.copy()
            signal.data += hp.data

            # 匹配滤波
            snr = matched_filter(hp, signal, psd=psd,
                                 low_frequency_cutoff=20.0)

            peak_snr = abs(snr).numpy().max()
            results.append((m1, m2, z, peak_snr))

            print(f"m1={m1:.1f}, m2={m2:.1f}, z={z:.2f}, SNR={peak_snr:.2f}") # 误报率的

基于Python的累积相位偏移建模

In [24]:
import numpy as np
from scipy.constants import G,c,pi
from scipy.integrate import quad
from astropy.cosmology import Planck18
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import minimize_scalar

In [20]:
# 宇宙学参数（采用Planck18的参数）
H0 = Planck18.H0.value
H0_si = H0 * 1e3 / 3.086e22   # 转换为SI单位，1/s
Omega_m = Planck18.Om0
Omega_lambda = 1 - Omega_m   # 内置参数不满足lambda + m = 1,故手动定义lambda = 1-m

In [32]:
# -----(1)宇宙膨胀导致的时延-----
def cosmological_time_delay(z):
    def integrand(z_prime):
        return 1.0 / ((1 + z_prime) * np.sqrt(Omega_m * (1 + z_prime)**3 + Omega_lambda))
    result, _ = quad(integrand, 0, z)
    return result / H0_si

# -----(2)暗物质晕的Shapiro时延-----

# --1.Millennium模拟暗物质晕数密度 n(M,z)的数据导入接口--
def load_nMz_interpolator(path):
    data = np.load(path)
    mass_bins = data['mass_bins']
    z_bins = data['z_bins']
    nMz_grid = data['nMz']
    # 二维插值
    interp_func = RegularGridInterpolator((mass_bins, z_bins), nMz_grid, bounds_error=False, fill_value=0.0)
    return interp_func

# --2.NFW剖面模型下的r_vir(M,z)定义--
def r_vir(M, H0_si, delta_c=200):
    rho_c = 3 * H0_si**2 / (8 * pi * G)   # 临界密度
    return (3 * M / (4 * pi * delta_c * rho_c))**(1/3)

# --3.shapiro_time_delay主函数
def shapiro_time_delay(z, frequency, phi_target=None, M_max=1e15, optimize_mmin=True, interp_nMz=None):
    assert interp_nMz is not None
    
    def integrand(M, z_):
        Rvir = r_vir(M, H0_si)
        n = interp_nMz([[z_, M]])[0]
        return (2 * G * M) / (c**3 * Rvir) * n
        
    # 寻找 M_min使偏差最小
    def rms_objective(M_min_candidate):
        result, _ = quad(integrand, M_min_candidate, M_max, args=(z,))
        delta_phi = 2 * pi * frequency *result
        return (delta_phi - phi_target)**2
        
    if optimize_mmin and phi_target is not None:
        opt = minimize_scalar(rms_objective, bounds=(1e10, 1e13), method='bounded')
        M_min_final = opt.x
    else:
        M_min_final = 1e11  # 默认下限

    # 积分
    result, _ = quad(intergrand, M_min_final, M_max, args=(z,), epsabs=1e-10)
    return result

# -----(3)LSS密度波动修正项（经验项）-----
def density_fluctuation_delay(z):
    return 1e-4 * z

# -----(4)累积相位偏移-----
def total_phase_shift(frequency, z, phi_target=None, M_max=1e15, optimize_mmin=True, interp_nMz=None):
    dt_cosmo = cosmological_time_delay(z)
    dt_shapiro = shapiro_time_delay(z, frequency, phi_target=None, M_max=1e15, optimize_mmin=True, interp_nMz=None)
    dt_density = density_fluctuation_delay(z)
    dt_total = dt_cosmo + dt_shapiro + dt_density
    delta_phi = 2 * pi * frequency * dt_total
    return delta_phi


累积相位偏移（关于z和f）的可视化

In [37]:
import matplotlib.pyplot as plt
freqs = np.linspace(10, 1000, 200)
zs = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
plt.figure(figsize=(10,6))
for z in zs:
    delta_phi_list = [total_phase_shift(frequency, z, phi_target=0.003, M_max=1e15, optimize_mmin=True, interp_nMz=load_nMz_interpolator('')) for frequency in freqs]
    plt.plot(freqs, delta_phi_list, label=f'z = {z}')
plt.xlabel('Frequency (Hz)')
plt.ylable('Δφ (radians)')
plt.title('Cumulative Phase Shift due to Cosmic Expansion and Weak Lensing')
plt.legend()
plt.grid()
plt.show()
plt.savefig('Cumulative Phase Shift due to Cosmic Expansion and Weak Lensing')